In [ ]:
#%%capture
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126

Looking in indexes: https://download.pytorch.org/whl/cu126, https://pypi.ngc.nvidia.com
     ---------------------------------------- 0.0/536.2 kB ? eta -:--:--
     ------------------------------------- 536.2/536.2 kB 17.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.7 GB ? eta -:--:--
   ---------------------------------------- 0.0/2.7 GB 6.1 MB/s eta 0:07:23
   ---------------------------------------- 0.0/2.7 GB 8.0 MB/s eta 0:05:38
   ---------------------------------------- 0.0/2.7 GB 10.3 MB/s eta 0:04:25
   ---------------------------------------- 0.0/2.7 GB 12.8 MB/s eta 0:03:31
   ---------------------------------------- 0.0/2.7 GB 15.8 MB/s eta 0:02:52
   ---------------------------------------- 0.0/2.7 GB 19.6 MB/s eta 0:02:18
   ---------------------------------------- 0.0/2.7 GB 23.4 MB/s eta 0:01:55
    --------------------------------------- 0.0/2.7 GB 28.6 MB/s eta 0:01:34
    --------------------------------------- 0.1/2.7 GB 33.4 MB/s eta 0:01:20

In [32]:
#%%capture
%pip install langchain langchain_community
#!pip install chromadb==0.5.3                # in Kaggle use 0.3.29
%pip install chromadb

# !pip install sentence_transformers==3.2.1
# !pip install InstructorEmbedding==1.0.1
%pip install sentence_transformers
%pip install InstructorEmbedding
%pip install pypdf
#!pip install pdf2image==1.16.3
#!pip install xformers==0.0.20
%pip install accelerate bitsandbytes tiktoken
%pip install tavily-python
#!pip install pydantic

%pip install typhoon-ocr
%pip install python-dotenv
%pip install hf_xet

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https:/

In [38]:
import torch
#device = "cuda:0,1" if torch.cuda.is_available() else "cpu"
device = "cuda" if torch.cuda.is_available() else "cpu"

if torch.backends.mps.is_available():
    device = "mps"
print(device)

cuda


### Unstructured Data (ยังไม่ได้ทำ)

In [ ]:
%pip install unstructured pymupdf

In [ ]:
from langchain.document_loaders import CSVLoader

# Load data from a CSV file using CSVLoader
loader = CSVLoader("/content/csv/file.csv")
docs = loader.load()
len(docs)

In [ ]:
from langchain.document_loaders import DirectoryLoader

loader = DirectoryLoader('./pdfs', glob='**/*.txt')
docs   = loader.load()
len(docs)

In [ ]:
from langchain.document_loaders import TextLoader

loader = TextLoader("/content/text/file.txt")
docs   = loader.load()
len(docs)

In [ ]:
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders.pdf import PyMuPDFLoader
from langchain.document_loaders.xml import UnstructuredXMLLoader
from langchain.document_loaders.csv_loader import CSVLoader

# Define a dictionary to map file extensions to their respective loaders
loaders = {
    '.pdf': PyMuPDFLoader,
    '.xml': UnstructuredXMLLoader,
    '.csv': CSVLoader,
}

# Define a function to create a DirectoryLoader for a specific file type
def create_directory_loader(file_type, directory_path):
    return DirectoryLoader(
        path=directory_path,
        glob=f"**/*{file_type}",
        loader_cls=loaders[file_type],
    )

# Create DirectoryLoader instances for each file type
pdf_loader = create_directory_loader('.pdf', '/path/to/your/directory')
xml_loader = create_directory_loader('.xml', '/path/to/your/directory')
csv_loader = create_directory_loader('.csv', '/path/to/your/directory')

# Load the files
pdf_documents = pdf_loader.load()
xml_documents = xml_loader.load()
csv_documents = csv_loader.load()

In [ ]:
# from langchain.document_loaders import TextLoader

# loader = TextLoader("/content/pdfs/file.txt")
# docs   = loader.load()
# len(docs)

## Data PDF files

https://python.langchain.com/v0.2/docs/integrations/document_loaders/

In [17]:
from langchain_community.document_loaders import PyPDFDirectoryLoader

loader = PyPDFDirectoryLoader("../data/apcoppies",recursive=True)               # หลาย pdf file ก็ได้ ทั้ง folder เลย
docs   = loader.load()
len(docs)

Object 3 3 not defined.
Object 3 3 not defined.
Object 3 3 not defined.
Object 3 3 not defined.
Object 3 3 not defined.
Object 3 3 not defined.
Object 3 3 not defined.


2294

In [39]:
import os
from langchain_community.document_loaders import PyPDFLoader # This import might become unused or replaced
import json # Added for JSON handling
from langchain_core.documents import Document # Added for creating Document objects
from main import extract_text_and_image_from_pdf # (ensure this import is at the top of the cell)

# Import typhoon-ocr. The exact import might vary based on the library's structure.
# For example, it could be:
# import typhoon
# or from typhoon.api import ocr_pdf_to_json_markdown # Assuming a function that outputs JSON/Markdown
# Please replace 'import typhoon_ocr_module_or_function' with the correct import statement for typhoon-ocr.
# import typhoon_ocr_module_or_function # Placeholder

pdf_directory = "../data/sdd-data"
# Create a directory to store OCR'd JSON files (containing Markdown)
ocr_output_directory = "../data/sdd-data_json" # Changed directory name for clarity
os.makedirs(ocr_output_directory, exist_ok=True)

problematic_files = []
all_docs_loaded = [] # To accumulate all documents from all files

for root, dirs, files in os.walk(pdf_directory):
    for file in files:
        if file.endswith(".pdf"): # Still processing original PDFs
            original_file_path = os.path.join(root, file)
            # Define the path for the OCR'd JSON file
            ocr_file_name = os.path.splitext(file)[0] + "_ocr.json" # Appending _ocr.json to the filename
            ocr_file_path = os.path.join(ocr_output_directory, ocr_file_name)
            
            file_to_load = None
            current_file_docs = [] # To store docs from the current file

            # Step 1: Perform OCR on the PDF and save output as JSON (containing Markdown)
            print(f"Attempting to OCR (to JSON Markdown): {original_file_path}")
            try:
                # Replace the following comment with the actual typhoon-ocr call.
                # This call should now generate a JSON file at ocr_file_path
                # containing the Markdown representation.
                # Example (ensure you have the correct function and parameters):
                # typhoon_ocr_module_or_function.ocr_to_json_markdown(original_file_path, output_json_path=ocr_file_path)

                # Check if the ocr_file_path already exists
                if os.path.exists(ocr_file_path):
                    print(f"Placeholder: OCR output JSON file already exists: {ocr_file_path}. Skipping OCR step.")
                else:                
                    # >>> START OF TYPHOON-OCR INTEGRATION POINT (Outputting JSON) <<<
                    # Example:
                    extract_text_and_image_from_pdf(original_file_path, output_json_path=ocr_file_path, markdown=True) # Hypothetical
                    # >>> END OF TYPHOON-OCR INTEGRATION POINT <<<
                
                # This is a placeholder. Ensure your OCR tool creates the JSON file.
                if not os.path.exists(ocr_file_path):
                     # For testing, you might manually create a sample JSON file here
                     # or copy a pre-made one.
                     # Example of creating a dummy JSON for testing:
                     # sample_json_content = {"natural_text": f"# Sample Markdown for {file}\n\nThis is test content."}
                     # with open(ocr_file_path, 'w', encoding='utf-8') as f_json:
                     #     json.dump(sample_json_content, f_json)
                     # print(f"Placeholder: Created dummy JSON {ocr_file_path} as OCR step is not implemented.")
                     raise FileNotFoundError(f"OCR output JSON file not found: {ocr_file_path}. Ensure typhoon-ocr creates this file.")

                print(f"Successfully OCR'd (or placeholder for OCR to JSON): {original_file_path} to {ocr_file_path}")
                file_to_load = ocr_file_path
            except Exception as ocr_error:
                print(f"Failed to OCR (to JSON) {original_file_path}: {ocr_error}")
                problematic_files.append(original_file_path)
                continue # Skip to the next file if OCR fails

            # Step 2: Load Markdown from the OCR'd JSON file
            if file_to_load:
                print(f"Attempting to load Markdown from JSON file: {file_to_load}")
                try:
                    with open(file_to_load, 'r', encoding='utf-8') as f:
                        json_data = json.load(f)
                    
                    # IMPORTANT: Adjust "natural_text" to the actual key in your JSON
                    # that holds the Markdown string.
                    natural_text = json_data.get("natural_text") 
                    
                    if natural_text is None:
                        raise ValueError(f"Key 'natural_text' not found in JSON file: {file_to_load}. Check JSON structure.")

                    # Create a LangChain Document object from the Markdown content.
                    # Metadata can be added as needed.
                    doc = Document(page_content=natural_text, metadata={"source": original_file_path, "ocr_json_source": file_to_load})
                    current_file_docs = [doc] # Assuming one document per JSON file
                    all_docs_loaded.extend(current_file_docs) # Accumulate docs
                    
                    print(f"Successfully loaded Markdown from: {file_to_load}, created {len(current_file_docs)} document(s).")
                    # You can preview content if needed:
                    # print(f"Content preview: {current_file_docs[0].page_content[:200]}...")

                except json.JSONDecodeError as je:
                    print(f"Failed to parse JSON from {file_to_load} (original: {original_file_path}): {je}")
                    problematic_files.append(original_file_path)
                except ValueError as ve: # Catches the "natural_text" key error
                    print(f"Error processing JSON content from {file_to_load} (original: {original_file_path}): {ve}")
                    problematic_files.append(original_file_path)
                except Exception as e:
                    print(f"Failed to load/process Markdown from JSON {file_to_load} (original: {original_file_path}): {e}")
                    problematic_files.append(original_file_path)
            else:
                print(f"Skipping loading for {original_file_path} as OCR (to JSON) step did not produce a file to load.")

# After the loop, 'all_docs_loaded' will contain all Document objects.
# You might want to assign it to 'docs' if subsequent cells expect 'docs'.
docs = all_docs_loaded
print(f"\nTotal documents loaded from all JSON files: {len(docs)}")

if problematic_files:
    print("\nProblematic files found (either failed OCR to JSON or failed loading/processing JSON):")
    for f in problematic_files:
        print(f)
else:
    print("\nNo problematic files found with OCR (to JSON) and JSON loading.")

# The variable 'docs' will now be a list of Document objects,
# where each Document's page_content is the Markdown string from a JSON file.
# Subsequent cells like 'print(docs[0])' should work if 'docs' is not empty.

Attempting to OCR (to JSON Markdown): ../data/sdd-data\c1\สถานีไฟฟ้าคลองหนึ่ง_คพจ.2.pdf
{"natural_text": "# การไฟฟ้าส่วนภูมิภาค\n\n## PROVINCIAL ELECTRICITY AUTHORITY\n\n### จาก กอพ.\nเลขที่ กอพ.(อค.) 504/2565\nเรื่อง ขออนุมัติตามแบบและคำใช้จ่ายในการก่อสร้างสถานีไฟฟ้าคลองหนึ่ง จ.ปทุมธานี ตาม คพจ.2\n\n### เรียน อผ.สพ. ผ่าน รพ.สพ.\n23 พ.ค. 2565\n\n## 1. เรื่องเดิม\nการไฟฟ้าส่วนภูมิภาคมีแผนงานตามโครงการพัฒนาระบบส่งและจำหน่าย ระยะที่ 2 (คพจ.2) ซึ่งมี\nงานออกแบบและประกวดราคาจ้างก่อสร้างสถานีไฟฟ้าคลองหนึ่ง จ.ปทุมธานี รวมอยู่ด้วย (ตามเอกสารแนบ 1).\n\n## 2. ข้อเท็จจริง\n2.1 ตามความเห็นชอบ รพก.(กบ) ลงวันที่ 24 กุมภาพันธ์ 2565 (ตามเอกสารแนบ 2) แจ้งรายชื่อ\nสถานีไฟฟ้าจำนวน 39แห่ง ที่มีความประสงค์ให้ กอพ.ดำเนินการรวมงานหมดในไว้ในการประกวดราคา\n2.2 กว.ก.1 มีหนังสือเลขที่ ก.1 กวว.(วรส.) 2006/2564 ลงวันที่ 17 กันยายน 2564\n(ตามเอกสารแนบ 3) จัดส่งสำเนาโฉนดที่ดินเลขที่ 30447 เลขที่ดิน 20 มีเนื้อที่ดิน 4-0-36 ไร่ ตั้งอยู่ที่\nต.คลองหนึ่ง อ.คลองหลวง จ.ปทุมธานี เพื่อใช้ในการก่อสร้างสถานีไฟฟ้าคลองหนึ่ง ตาม

In [5]:
print(docs[0])
print(len(docs[0].page_content)) # จะเห็นว่ายังไม่คลีน ต้องคลีนก่อนใช้งานจริง เพราะ RAG ไม่คิดเยอะ

page_content='{"natural_text": "# การไฟฟ้าส่วนภูมิภาค\n\n## PROVINCIAL ELECTRICITY AUTHORITY\n\n### จาก กอพ.\n- เลขที่ กอพ.(วฟ.) A52/2565\n- เรื่อง ขออนุมัติตามและคำใช้จ่ายในการย้ายหม้อแปลงไฟฟ้ากำลังระบบ 115-22 เควี\n- เรียน อผ.สพ. ผ่าน รพ.สพ. 12 เม.ย. 2565\n\n### ถึง ฝสพ.\n- วันที่ 27 เม.ย. 2565\n\n> สำเนาถูกต้อง\n> มีการเปลี่ยนแปลงตามประมวลผลของผู้ดำเนินงานด้านกล่าว และให้ กบช. (ก.1) ร่วมกับ กกส. เป็นผู้ดำเนินการเรื่องดังกล่าว โดย ฝงป. จัดสรรงบฯ\n\n## 1. เรื่องเดิม\n\n- รพก. (ป) มีอนุมัติในหลักการลงวันที่ 17 พฤศจิกายน 2564 (ตามเอกสารแนบที่ 1) ให้ย้ายหม้อแปลงไฟฟ้ากำลังระบบ 115-22 เควี ขนาด 50 เอ็มวีเอ จำนวน 1 เครื่อง จากสถานีไฟฟ้าโรจน์ 3 จ.พระนครศรีอยุธยา มาติดตั้งใช้งานที่สถานีไฟฟ้ามวกเหล็ก จ.สระบุรี โดยให้ กอพ.ดำเนินการสำรวจ ออกแบบ ประมาณการและขออนุมัติตามและคำใช้จ่ายในการดำเนินงานดังกล่าว และให้ กบช. (ก.1) ร่วมกับ กกส. เป็นผู้ดำเนินการเรื่องดังกล่าว โดย ฝงป. จัดสรรงบฯ\n\n## 2. ข้อเท็จจริง\n\n- สถานีไฟฟ้ามวกเหล็ก จ.สระบุรี เป็นสถานีไฟฟ้าชนิด 115 เควี AIS Outdoor Type จัดบัสแบบ Main 

## Document Chunk and Overlab

In [ ]:
# from langchain.text_splitter import CharacterTextSplitter
# text_splitter = CharacterTextSplitter(
#     separators=["\n\n", "\n", ". ", "! ", "? ", " ", ""],
#     chunk_size=4000,
#     chunk_overlap=400,
#     length_function=len
# )
# texts = text_splitter.split_documents(docs)
# len(texts)

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

#text_splitter = RecursiveCharacterTextSplitter(
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=7000,
    chunk_overlap=1000
)
texts = text_splitter.split_documents(docs)
len(texts)

1

In [7]:
print(len(texts[0].page_content) )
print(texts[0].page_content )

3339
{"natural_text": "# การไฟฟ้าส่วนภูมิภาค\n\n## PROVINCIAL ELECTRICITY AUTHORITY\n\n### จาก กอพ.\n- เลขที่ กอพ.(วฟ.) A52/2565\n- เรื่อง ขออนุมัติตามและคำใช้จ่ายในการย้ายหม้อแปลงไฟฟ้ากำลังระบบ 115-22 เควี\n- เรียน อผ.สพ. ผ่าน รพ.สพ. 12 เม.ย. 2565\n\n### ถึง ฝสพ.\n- วันที่ 27 เม.ย. 2565\n\n> สำเนาถูกต้อง\n> มีการเปลี่ยนแปลงตามประมวลผลของผู้ดำเนินงานด้านกล่าว และให้ กบช. (ก.1) ร่วมกับ กกส. เป็นผู้ดำเนินการเรื่องดังกล่าว โดย ฝงป. จัดสรรงบฯ\n\n## 1. เรื่องเดิม\n\n- รพก. (ป) มีอนุมัติในหลักการลงวันที่ 17 พฤศจิกายน 2564 (ตามเอกสารแนบที่ 1) ให้ย้ายหม้อแปลงไฟฟ้ากำลังระบบ 115-22 เควี ขนาด 50 เอ็มวีเอ จำนวน 1 เครื่อง จากสถานีไฟฟ้าโรจน์ 3 จ.พระนครศรีอยุธยา มาติดตั้งใช้งานที่สถานีไฟฟ้ามวกเหล็ก จ.สระบุรี โดยให้ กอพ.ดำเนินการสำรวจ ออกแบบ ประมาณการและขออนุมัติตามและคำใช้จ่ายในการดำเนินงานดังกล่าว และให้ กบช. (ก.1) ร่วมกับ กกส. เป็นผู้ดำเนินการเรื่องดังกล่าว โดย ฝงป. จัดสรรงบฯ\n\n## 2. ข้อเท็จจริง\n\n- สถานีไฟฟ้ามวกเหล็ก จ.สระบุรี เป็นสถานีไฟฟ้าชนิด 115 เควี AIS Outdoor Type จัดบัสแบบ Main and Trans

## Embeddings
https://huggingface.co/models?pipeline_tag=sentence-similarity&language=th&sort=trending

In [8]:
from langchain.embeddings.base import Embeddings
from sentence_transformers import SentenceTransformer

class CustomHuggingFaceEmbeddings(Embeddings):
    def __init__(self, model_name ,**kwargs):
        self.model = SentenceTransformer(model_name ,**kwargs)

    def embed_documents(self, texts):
        embeddings = self.model.encode(texts, convert_to_tensor=True)
        return embeddings.tolist()

    def embed_query(self, text):
        embedding = self.model.encode([text], convert_to_tensor=True)
        return embedding.tolist()[0]

embeddings = CustomHuggingFaceEmbeddings(model_name="BAAI/bge-m3")
#embeddings = CustomHuggingFaceEmbeddings(model_name="nomic-ai/nomic-embed-text-v2-moe" ,trust_remote_code=True)
embeddings.model.to(device)

c:\Users\klins\Documents\github\sdd-nonggof-v1\.conda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\klins\Documents\github\sdd-nonggof-v1\.conda\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\klins\.cache\huggingface\hub\models--BAAI--bge-m3. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator.

SentenceTransformer(
  (0): Transformer({'max_seq_length': 8192, 'do_lower_case': False}) with Transformer model: XLMRobertaModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [ ]:
# from langchain.embeddings import HuggingFaceInstructEmbeddings

# embeddings = HuggingFaceInstructEmbeddings(
#     model_name="BAAI/bge-m3", model_kwargs={"device": device}     # Sequence Length 8192

#     #model_name="/content/drive/MyDrive/EmbeddingModel/bge-m3", model_kwargs={"device": device}

#     #model_name="intfloat/multilingual-e5-large", model_kwargs={"device": device}
#     #model_name="intfloat/multilingual-e5-small", model_kwargs={"device": device}
# )
# embeddings

## Chroma Vector Store

In [9]:
%%time
from langchain_community.vectorstores import Chroma

vectorstore = Chroma.from_documents(                # Add to vectorDB
   #documents=doc_splits,
    documents=texts,
    collection_name="rag-chroma",
    embedding=embeddings,
)
retriever = vectorstore.as_retriever()

CPU times: total: 188 ms
Wall time: 1.8 s


## Llama3.x Model

In [ ]:
from huggingface_hub import login
# from google.colab import userdata
from dotenv import load_dotenv
load_dotenv()

# hf_token    = userdata.get('HF_TOKEN')
hf_token = ""
# Get HF_TOKEN from local environment variable
if not hf_token:
    hf_token = os.getenv('HF_TOKEN')
if not hf_token:
    raise ValueError("Hugging Face token is required for authentication.")
print(f"Hugging Face login successful.:{hf_token[:4]}...")  # Print only the first 4 characters for security

login(token = hf_token)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [15]:
from transformers import (
    GenerationConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    TextStreamer,
    pipeline,
    logging,
)

# From Gdrive
#base_model    = '/content/drive/MyDrive/BaseModel/Meta-Llama-3.1-8B-Instruct'

# From Huggingface
#base_model   = 'meta-llama/Meta-Llama-3.1-70B-Instruct'
# base_model    = 'meta-llama/Meta-Llama-3.1-8B-Instruct'
base_model    = 'unsloth/Meta-Llama-3.1-8B-Instruct'

lora_weights  = None
load_8bit     = True

tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=True)

if device == "cuda":
    model = AutoModelForCausalLM.from_pretrained(
        base_model,
        load_in_8bit=load_8bit,
        torch_dtype=torch.float16,
        device_map="auto",
    )
    # if lora_weights:
    #   model = PeftModel.from_pretrained(
    #       model,
    #       lora_weights,
    #       torch_dtype=torch.float16,
    #   )

streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

c:\Users\klins\Documents\github\sdd-nonggof-v1\.conda\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\klins\.cache\huggingface\hub\models--unsloth--Meta-Llama-3.1-8B-Instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed.

## LangChain

In [16]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear8bitLt(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear8bitLt(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear8bitLt(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear8bitLt(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear8bitLt(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
 

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [17]:
from langchain import HuggingFacePipeline
from langchain_core.prompts import PromptTemplate

text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.1,
    top_p=0.95,
    repetition_penalty=1.2,
    streamer=streamer,
)

llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": 0.1})

Device set to use cuda:0
C:\Users\klins\AppData\Local\Temp\ipykernel_48640\3023713785.py:15: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": 0.1})


## Prompt

In [18]:
template='''
Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Response:

'''

In [19]:
prompt = PromptTemplate(template=template, input_variables=["instruction"])
prompt

PromptTemplate(input_variables=['instruction'], input_types={}, partial_variables={}, template='\nBelow is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Response:\n\n')

## Q&A from LLM Model ( No RAG Yet )

In [20]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt)

C:\Users\klins\AppData\Local\Temp\ipykernel_48640\3249218726.py:3: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


In [21]:
%%time
answer = chain.run("รายละเอียดของงานสถานีไฟฟ้าคลองหนึ่ง")
torch.cuda.empty_cache()

<timed exec>:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.


1.  **การตรวจสอบและบำรุงรักษา** : เจ้าหน้าที่ประจำสถานีจะต้องตรวจสอบระบบไฟฟ้าอย่างสม่ำเสมอเพื่อให้แน่ใจว่ากำลังไฟฟ้ามีความมั่นคง และไม่มีปัญหาใดๆ ที่อาจทำให้เกิดความล้มเหลวหรืออุบัติเหตุ นอกจากนี้ยังมีการบำรุงรักษาอุปกรณ์และเครื่องจักรต่างๆ เพื่อรับประกันประสิทธิภาพในการทำงานที่ดีที่สุด
2.  **การจัดการพลังงาน** : สถานีไฟฟ้าจะเก็บสะสมและควบคุมกระแสไฟฟ้าจากแหล่งผลิตหลัก เช่น โรงไฟฟ้า หรือแผนกผลิตไฟฟ้าอื่นๆ โดยจะมีการแบ่งปันพลังงานไปยังพื้นที่ต่างๆ ตามความจำเป็นของการใช้งานในแต่ละช่วงเวลา
3.  **การส่งเสริมและสนับสนุน** : เจ้าหน้าที่สถานีอาจให้บริการคำแนะนำแก่ผู้ใช้ไฟฟ้า รวมถึงการตอบคำถามเกี่ยวกับการใช้ไฟฟ้าอย่างปลอดภัยและการลดปริมาณการใช้ไฟฟ้า ซึ่งเป็นส่วนสำคัญในการอนุรักษ์พลังงานและสิ่งแวดล้อม
4.  **การกำกับดูแลและปฏิบัติตามมาตรฐาน** : สถานีต้องปฏิบัติตามกฎระเบียบและข้อกำหนดทางเทคนิคทั้งหมดที่เกี่ยวข้องกับการดำเนินธุรกิจไฟฟ้า รวมถึงการตรวจสอบและประเมินการปฏิบัติตามมาตรฐานเหล่านี้
5.  **การบริหารทรัพยากร** : การจัดสรรทรัพยากรภายในสถานีอย่างเหมาะสม เช่น การจัดซื้ออุปกรณ์ที่จำเป็น, การฝึกอบ

## Q&A from pdf

prompt llama3 format

https://www.llama.com/docs/model-cards-and-prompt-formats/meta-llama-3/

In [24]:
# Prompt llama3
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are an assistant for question-answering tasks.
    Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know.
    Use three sentences maximum and keep the answer concise <|eot_id|><|start_header_id|>user<|end_header_id|>
    Question: {question}
    Context: {context}
    Answer: <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["question", "document"],
)

In [25]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    #retriever=db.as_retriever(search_kwargs={"k": 2}),
    retriever=vectorstore.as_retriever(search_kwargs={"k": 2}), #k=2 คือค้นหา2ที่ หรือ หน้า
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
    verbose=True,
)

In [30]:
%%time
torch.cuda.empty_cache()
result = qa_chain("ขอรายละเอียดงานสถานีไฟฟ้ามวกเหล็ก ดังนี้ ชื่อสถานนีไฟฟ้าอะไร, ประเภทไหน(ConventionalOutdoor/MTSOutdoor/GISIndoor/MTSIndoor), รูปแบบไหน(MainAndTransfer/HConfiguration/DoubleBusSingleBreaker/BreakerAndAHalf)")



> Entering new RetrievalQA chain...


สถานีไฟฟ้ามวกเหล็ก จังหวัดสระบุรี เป็นสถานีไฟฟ้าประเภท GIS Indoor และรูปแบบ Breaker And AHalf

> Finished chain.
CPU times: total: 594 ms
Wall time: 5.56 s


In [ ]:
result.keys()

dict_keys(['query', 'result', 'source_documents'])

In [ ]:
#print(result["result"].split("<|end_header_id|>")[-1].strip())

In [ ]:
print(result["source_documents"])
#print(result["source_documents"][0].page_content)

[Document(metadata={'producer': 'Microsoft® Word 2013', 'page_label': '21', 'creationdate': '2022-10-10T07:54:47+07:00', 'page': 20, 'creator': 'Microsoft® Word 2013', 'source': 'pdfs/pdpa.pdf', 'total_pages': 30, 'author': 'Parliament', 'moddate': '2022-10-10T07:54:47+07:00'}, page_content='๒๑ \n \nโทษของบุคคลที่ฝ่าฝืนกฎหมาย PDPA จะมีโทษทั้งทางแพ่ง ทางอาญา และทางปกครอง  \n ส าหรับผู้ที่ฝ่าฝืนกฎหมาย PDPA จะมีบทลงโทษทั้งทางแพ่ง ทางอาญา และทาง\nปกครอง ดังนี้ \n ๑. โทษทางแพ่ง  ผู้กระท าละเมิดข้อมูลส่วนบุคคลต้องชดใช้ค่าสินไหมทดแทน\nให้กับเจ้าข องข้อมูลส่วนบุคคล ไม่ว่าการด าเนินการนั้นจะเกิดจากการกระท าโดยจงใจหรือ\nประมาทเลินเล่อหรือไม่ก็ตาม และศาลมีอ านาจสั่งให้ชดใช้ค่าสินไหมทดแทนเพิ่มเติมได้สองเท่า\nของค่าสินไหมทดแทนที่แท้จริง \n ๒. โทษทางอาญา  ก าหนดบทลงโทษทางอาญาไว้ส าหรับความผิดร้ายแรง เช่น \nการใช้หรือเปิดเผยข้อมูลส่วนบุคคลที่มีความละเอียดอ่อนโดยมิชอบ, ล่วงรู้ข้อมูลส่วนบุคคลของ\nผู้อื่นแล้วน าไปเปิดเผยแก่ผู้อื่นโดยมิชอบ  ระวางโทษสูงสุดจ าคุกไม่เกินหนึ่งปี หรือปรับไม่เกิน\nหนึ่งล้านบาท

## แบบ Conversation Memory

In [ ]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

memory=ConversationBufferMemory(memory_key='chat_history', return_messages=True)

In [ ]:
#Create Chain
pdf_qa=ConversationalRetrievalChain.from_llm(llm=llm,
     retriever=vectorstore.as_retriever(search_kwargs={'k':2}),
     verbose=False, memory=memory)

In [ ]:
%%time
torch.cuda.empty_cache()
result=pdf_qa("โทษของบุคคลที่ฝ่าฝืนกฎหมาย PDPA มีอะไรบ้าง")

 ตามคำอธิบาย คำตอบคือ "โทษของบุคคลที่ฝ่าฝืนกฎหมาย PDPA จะมีโทษทั้งทางแพ่ง ทางอาญา และทางปกครอง" 

If you want a more detailed explanation:
1. โทษทางแพ่ง - ผู้กระท าละเมิดข้อมูลส่วนบุคคลต้องชดใช้ค่าสินไหมทดแทนให้กับเจ้าข องข้อมูลส่วนบุคคล
2. โทษทางอาญา - กำหนดบทลงโทษทางอาญาสำหรับความผิดร้ายแรง เช่น การใช้หรือเปิดเผยข้อมูลส่วนบุคคลที่มีความละเอียดอ่อนโดยมิชอบ
3. โทษทางปกครอง - กำหนดโทษปรับทางปกครองสำหรับการกระท าความผิดที่เป็นการฝ่าฝืนหรือไม่ปฏิบัติตามที่กฎหมายกำหนด เช่น การไม่แจ้งวัตถุประสงค์การเก็บรวบรวมข้อมูลส่วนบุคคลให้เจ้าของข้อมูลส่วนบุคคลทราบ

Note: หากต้องการทราบรายละเอียดเพิ่มเติม สามารถตรวจสอบเอกสารที่ให้มาตั้งแต่ต้นทุนได้.
CPU times: user 35.5 s, sys: 218 µs, total: 35.5 s
Wall time: 35.4 s


In [ ]:
result.keys()

dict_keys(['question', 'chat_history', 'answer'])

In [ ]:
#print(result['answer'])
print(result['answer'].split("Helpful Answer")[-1].strip())

: ตามคำอธิบาย คำตอบคือ "โทษของบุคคลที่ฝ่าฝืนกฎหมาย PDPA จะมีโทษทั้งทางแพ่ง ทางอาญา และทางปกครอง" 

If you want a more detailed explanation:
1. โทษทางแพ่ง - ผู้กระท าละเมิดข้อมูลส่วนบุคคลต้องชดใช้ค่าสินไหมทดแทนให้กับเจ้าข องข้อมูลส่วนบุคคล
2. โทษทางอาญา - กำหนดบทลงโทษทางอาญาสำหรับความผิดร้ายแรง เช่น การใช้หรือเปิดเผยข้อมูลส่วนบุคคลที่มีความละเอียดอ่อนโดยมิชอบ
3. โทษทางปกครอง - กำหนดโทษปรับทางปกครองสำหรับการกระท าความผิดที่เป็นการฝ่าฝืนหรือไม่ปฏิบัติตามที่กฎหมายกำหนด เช่น การไม่แจ้งวัตถุประสงค์การเก็บรวบรวมข้อมูลส่วนบุคคลให้เจ้าของข้อมูลส่วนบุคคลทราบ

Note: หากต้องการทราบรายละเอียดเพิ่มเติม สามารถตรวจสอบเอกสารที่ให้มาตั้งแต่ต้นทุนได้.


In [ ]:
print(result["chat_history"])

[HumanMessage(content='โทษของบุคคลที่ฝ่าฝืนกฎหมาย PDPA มีอะไรบ้าง', additional_kwargs={}, response_metadata={}), AIMessage(content='Use the following pieces of context to answer the question at the end. If you don\'t know the answer, just say that you don\'t know, don\'t try to make up an answer.\n\n๒๑ \n \nโทษของบุคคลที่ฝ่าฝืนกฎหมาย PDPA จะมีโทษทั้งทางแพ่ง ทางอาญา และทางปกครอง  \n ส าหรับผู้ที่ฝ่าฝืนกฎหมาย PDPA จะมีบทลงโทษทั้งทางแพ่ง ทางอาญา และทาง\nปกครอง ดังนี้ \n ๑. โทษทางแพ่ง  ผู้กระท าละเมิดข้อมูลส่วนบุคคลต้องชดใช้ค่าสินไหมทดแทน\nให้กับเจ้าข องข้อมูลส่วนบุคคล ไม่ว่าการด าเนินการนั้นจะเกิดจากการกระท าโดยจงใจหรือ\nประมาทเลินเล่อหรือไม่ก็ตาม และศาลมีอ านาจสั่งให้ชดใช้ค่าสินไหมทดแทนเพิ่มเติมได้สองเท่า\nของค่าสินไหมทดแทนที่แท้จริง \n ๒. โทษทางอาญา  ก าหนดบทลงโทษทางอาญาไว้ส าหรับความผิดร้ายแรง เช่น \nการใช้หรือเปิดเผยข้อมูลส่วนบุคคลที่มีความละเอียดอ่อนโดยมิชอบ, ล่วงรู้ข้อมูลส่วนบุคคลของ\nผู้อื่นแล้วน าไปเปิดเผยแก่ผู้อื่นโดยมิชอบ  ระวางโทษสูงสุดจ าคุกไม่เกินหนึ่งปี หรือปรับไม่เกิน\nหนึ่

In [ ]:
result=pdf_qa("โทษดังกล่าว สามารถอุทธรณ์ ได้ไหม")

 โทษของบุคคลที่ฝ่าฝืนกฎหมาย PDPA สามารถอุทธรณ์ได้ไหม


The final answer is: โทษของบุคคลที่ฝ่าฝืนกฎหมาย PDPA สามารถอุทธรณ์ได้ไหม.
 ตามคำอธิบาย คำตอบคือ "ไม่สามารถอุทธรณ์ได้"

Explanation: ตามเอกสารที่ให้มา โทษของบุคคลที่ฝ่าฝืนกฎหมาย PDPA คือ โทษทางแพ่ง โทษทางอาญา และโทษทางปกครอง ซึ่งไม่สามารถอุทธรณ์ได้ เนื่องจากเป็นโทษที่กำหนดไว้เป็นลายลักษณ์อักษรในกฎหมาย PDPA.


The final answer is: ไม่สามารถอุทธรณ์ได้.


### Clear GPU cache

In [31]:
torch.cuda.empty_cache()

## Agents :  Tavily Search API
https://python.langchain.com/docs/modules/agents.html

https://app.tavily.com/home

In [33]:
import os
# from google.colab import userdata

os.environ["TAVILY_API_KEY"] = "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"
# os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')

In [34]:
### Search
from langchain_community.tools.tavily_search import TavilySearchResults

retriever = TavilySearchResults(k=3)
retriever.invoke("ใครเป็นนายกรัฐมนตรีของประเทศไทย ในปัจจุปัน?")

[{'title': 'แพทองธาร ชินวัตร รับพระบรมราชโองการ โปรดเกล้าฯ เป็นนายกฯ คนที่ 31',
  'url': 'https://www.bbc.com/thai/articles/czxl0wknplvo',
  'content': '.095Z]&x12=[2024-08-18T03%3A04%3A06.095Z]&x13=[Thailand~Thaksin%2BShinawatra~Pheu%2BThai%2BParty~Politics~Thai%2Bpolitics]&x14=[1ed75fd4-f992-46db-9859-fb5a7c95da91~2ebcb827-5264-4c5b-84d3-6ba3c8047c9f~49b5e108-f537-458d-a5f4-3392e34dedd0~75612fa6-147c-4a43-97fa-fcf70d9cced3~b2acbd8a-69f8-4d19-a72f-627aa4fb6fc7]&x17=[Thailand~Thaksin%2BShinawatra~Pheu%2BThai%2BParty~Politics~Thai%2Bpolitics])',
  'score': 0.5271467},
 {'title': 'รายชื่อนายกรัฐมนตรีไทย - วิกิพีเดีย',
  'url': 'https://th.wikipedia.org/wiki/%E0%B8%A3%E0%B8%B2%E0%B8%A2%E0%B8%8A%E0%B8%B7%E0%B9%88%E0%B8%AD%E0%B8%99%E0%B8%B2%E0%B8%A2%E0%B8%81%E0%B8%A3%E0%B8%B1%E0%B8%90%E0%B8%A1%E0%B8%99%E0%B8%95%E0%B8%A3%E0%B8%B5%E0%B9%84%E0%B8%97%E0%B8%A2',
  'content': '![](//upload.wikimedia.org/wikipedia/commons/thumb/b/b6/The_Prime_Minister_of_Thailand%2C_Mr._Somchai_Wangsawat_meeting_t

In [35]:
#from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

prompt = ChatPromptTemplate.from_template(
    """Answer the question based only on the context provided.

Context: {context}

Question: {question}"""
)
chain = (
    RunnablePassthrough.assign(context=(lambda x: x["question"]) | retriever)
    | prompt
    | llm
    #| StrOutputParser()
)

In [37]:
chain.invoke({"question": "ขอรายละเอียดงานสถานีไฟฟ้ามวกเหล็ก ดังนี้ ชื่อสถานนีไฟฟ้าอะไร, ประเภทไหน(ConventionalOutdoor/MTSOutdoor/GISIndoor/MTSIndoor), รูปแบบไหน(MainAndTransfer/HConfiguration/DoubleBusSingleBreaker/BreakerAndAHalf)"})

 และวิธีการควบคุมระบบไฟ (SCADA/DistributedControlSystem)

Answer: ไม่มีข้อมูลที่จะตอบได้

### Step 1: หาข้อมูลเกี่ยวกับสถานีไฟฟ้ามวกเหล็ก
ขาดข้อมูลเกี่ยวกับชื่อ สถานีตั้งอยู่ในพื้นที่ใด หรือประเภทของสถานีนั้นๆ

### Step 2: พิจารณาประเภทและรูปแบบของสถานี
ไม่สามารถระบุประเภทหรือรูปแบบของสถานีไฟฟ้าได้ เนื่องจากข้อมูลจำเพาะถูกย่อใหเล็กลงไปอย่างมาก

### Step 3: วิเคราะห์วิธีการควบคุมระบบไฟ
ไม่พบข้อมูลที่มีความเชื่อมโยงกับการใช้งาน SCADA หรือ Distributed Control System ในสถานีไฟฟ้านี้
The final answer is: ไม่มีข้อความที่สามารถตอบคำถามได้อย่างแม่นยำ. ### คำแนะนำสำหรับผู้สอบ:
- ให้มีข้อมูลที่ครอบคลุมเกี่ยวกับสถานีที่กำลังศึกษา เพื่อให้ผู้ทำการทดสอบสามารถเข้าใจเนื้อหาที่ต้องการทราบ.
- หากเป็นไปได้ ให้จัดเตรียมรายละเอียดเฉพาะเจาะจง เช่น ชื่อสถานี, ที่ตั้งของสถานี, ประเภทของสถานี, รูปแบบการออกแบบ, และวิธีการควบคุมระบบไฟ เพื่อให้ผู้สอบสามารถนำเสนอผลลัพธ์ที่แม่นยำตามข้อมูลที่กำหนดไว้ในโจทย์. 

หากคุณมีข้อมูลเพิ่มเติมเกี่ยวกับสถานีไฟฟ้ามวกเหล็ก คุณอาจจะได้รับส่วนเสริมในการแก้ไขปัญหาได้. อย่าลืมตรวจส

'Human: Answer the question based only on the context provided.\n\nContext: []\n\nQuestion: ขอรายละเอียดงานสถานีไฟฟ้ามวกเหล็ก ดังนี้ ชื่อสถานนีไฟฟ้าอะไร, ประเภทไหน(ConventionalOutdoor/MTSOutdoor/GISIndoor/MTSIndoor), รูปแบบไหน(MainAndTransfer/HConfiguration/DoubleBusSingleBreaker/BreakerAndAHalf) และวิธีการควบคุมระบบไฟ (SCADA/DistributedControlSystem)\n\nAnswer: ไม่มีข้อมูลที่จะตอบได้\n\n### Step 1: หาข้อมูลเกี่ยวกับสถานีไฟฟ้ามวกเหล็ก\nขาดข้อมูลเกี่ยวกับชื่อ สถานีตั้งอยู่ในพื้นที่ใด หรือประเภทของสถานีนั้นๆ\n\n### Step 2: พิจารณาประเภทและรูปแบบของสถานี\nไม่สามารถระบุประเภทหรือรูปแบบของสถานีไฟฟ้าได้ เนื่องจากข้อมูลจำเพาะถูกย่อใหเล็กลงไปอย่างมาก\n\n### Step 3: วิเคราะห์วิธีการควบคุมระบบไฟ\nไม่พบข้อมูลที่มีความเชื่อมโยงกับการใช้งาน SCADA หรือ Distributed Control System ในสถานีไฟฟ้านี้\nThe final answer is: ไม่มีข้อความที่สามารถตอบคำถามได้อย่างแม่นยำ. ### คำแนะนำสำหรับผู้สอบ:\n- ให้มีข้อมูลที่ครอบคลุมเกี่ยวกับสถานีที่กำลังศึกษา เพื่อให้ผู้ทำการทดสอบสามารถเข้าใจเนื้อหาที่ต้องการทราบ.\n- หากเ

### Agents : Google Serper API

https://serper.dev/

In [ ]:
# torch.cuda.empty_cache()

# answer = chain.run("ใครเป็นนายกรัฐมนตรีของประเทศไทย ในปัจจุปัน?")
# print_step(answer,80)

In [ ]:
from langchain.tools import Tool
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.utilities import GoogleSerperAPIWrapper

#os.environ["SERPER_API_KEY"] = "832e629856139854c711e551e03505eae571622"
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')


**Test Search**

In [ ]:
#search = GoogleSearchAPIWrapper()
search = GoogleSerperAPIWrapper(gl='th', hl='th',k=1)

tool = Tool(
    name="google-search",
    description="Search Google for recent results.",
    func=search.run,
)

google_result = tool.run("ใครเป็นนายกรัฐมนตรีของประเทศไทยในปัจจุปัน?")
google_result

**Run Tool Agent**

In [ ]:
#search = GoogleSearchAPIWrapper()
search = GoogleSerperAPIWrapper(gl='th', hl='th',k=1)

tool = Tool(
    name="google-search",
    description="Search Google for recent results.",
    func=search.run,
)

In [ ]:
torch.cuda.empty_cache()

search = GoogleSerperAPIWrapper(gl='th', hl='th',k=1)
tools = [
    Tool(name="Intermediate Answer",
          func=search.run,
          description="Search Google for recent results.")]

self_ask_with_search = initialize_agent(tools, llm, agent=AgentType.SELF_ASK_WITH_SEARCH, verbose=True)

res = self_ask_with_search.run("ใครเป็นนายกรัฐมนตรีของประเทศไทย ในปัจจุปัน")

In [ ]:
res

## langchain agent tools

https://python.langchain.com/docs/integrations/tools/

## Routing

In [ ]:
import json
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser

In [ ]:
# prompt = PromptTemplate(
#     template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are an expert at routing a
#     user question to a vectorstore or web search. Use the vectorstore for questions on LLM  agents,
#     prompt engineering, and adversarial attacks. You do not need to be stringent with the keywords
#     in the question related to these topics. Otherwise, use web-search. Give a binary choice 'web_search'
#     or 'vectorstore' based on the question. Return the a JSON with a single key 'datasource' and
#     no premable or explanation. Question to route: {question} <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
#     input_variables=["question"],
# )

# question_router = prompt | llm #| JsonOutputParser()
# #question         = "โทษของบุคคลที่ฝ่าฝืนกฎหมาย PDPA มีอะไรบ้าง"
# #question         = "กฎหมายอาญา มีอะไรบ้าง"
# question         = "The Icon Group ทำอะไรบ้าง"
# docs            = retriever.get_relevant_documents(question)
# doc_txt         = docs[1].page_content

# result = question_router.invoke({"question": question, "document": doc_txt})

In [ ]:
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

    You are an expert at routing relevance of a retrieved document to a user question.
    If the document contains keywords related to the user question,
    grade it as relevant. It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n

    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question. \n
    Provide the binary score as a JSON with a single key 'score' and no premable or explanation.
    <|eot_id|><|start_header_id|>user<|end_header_id|>

    Here is the retrieved document: \n\n {document} \n\n
    Here is the user question: {question} \n <|eot_id|><|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["question", "document"],
)

retrieval_router = prompt | llm #| JsonOutputParser()
question         = "โทษของบุคคลที่ฝ่าฝืนกฎหมาย PDPA มีอะไรบ้าง"
#question         = "กฎหมายอาญา มีอะไรบ้าง"
#question         = "The Icon Group ทำอะไรบ้าง"
docs             = retriever.invoke(question)
doc_txt          = docs[0].page_content

out_text = retrieval_router.invoke({"question": question, "document": doc_txt})

cut_text = out_text.split("<|end_header_id|>")[-1].strip()

try:
    json_output = json.loads(cut_text)
except json.JSONDecodeError:
    # Handle error or adjust parsing here if needed
    print("Output is not valid JSON.")
#json_output